In [1]:
using Multilane
using POMDPs
using POMDPToolbox
using MCTS
using Distributions # for Normal
using DataArrays
using DataFrames
using DataFramesMeta

#using Interact
using Interact
using Reactive
#using Records
using AutoViz
using AutomotiveDrivingModels
using JLD
#using Cairo
#using Reel
#using GenerativeModels

In [2]:
#Set up problem configuration
nb_lanes = 4
pp = PhysicalParam(nb_lanes,lane_length=450.) #2.=>col_length=8
_discount = 0.9
nb_cars=15

#rmodel = NoCrashRewardModel()
rmodel  = NoCrashRewardModel(0.0,0,2.5,1,100.,4,2500,100)
#rmodel  = NoCrashRewardModel(0.0,0,2.5,1,0.0,4,2500,100)
#be = standard_uniform()
#dmodel = NoCrashIDMMOBILModel(nb_cars, pp, behaviors = be)
dmodel = NoCrashIDMMOBILModel(nb_cars, pp)

mdp = NoCrashMDP{NoCrashRewardModel}(dmodel, rmodel, _discount)

rng = MersenneTwister(5);

In [3]:
is = relaxed_initial_state(mdp);
#CurrentState = is;

In [4]:
@async begin
    udpsock = UDPSocket()
    sock = UDPSocket()
    CurrentState = is;
    bind(udpsock, ip"127.0.0.1" , 2399)
    while true
        strvec = String(copy(recv(udpsock)))
        i = search(strvec,',',1)
        str1 = strvec[start(strvec):i-1]
        println(str1)
        k = search(strvec,',',i+1)
        str2 = strvec[i+1:k-1]
        println(str2)
        j = search(strvec,'\n',1)
        strt = strvec[k+1:j-1]
        println(strt)
        num1 = parse(Float64,str1)
        num2 = parse(Float64,str2)
        numt = parse(Float64,strt)
        MKZAction = MLAction(num1,num2)
        CurrentState=relaxed_initial_state_TRI(mdp,CurrentState,MKZAction,numt);
        println(CurrentState.x)
        println(CurrentState.cars[1].y)
        send(sock,ip"127.0.0.1",2398, string(string(CurrentState.x),",",string(CurrentState.cars[1].y),",end \n"))
    end
end

Task (runnable) @0x00007f01fe75e4d0

0
0
0.5
4648.202852362261
4.0
0
0
0.5
4671.839562225394
4.0
0
0
0.5
4695.4762720885265
4.0
1
0
0.75
4719.675481951659
4.0
1
0
0.75
4744.437191814792
4.0


In [ ]:
MKZAction = MLAction(0.0,0.0)
CurrentState=relaxed_initial_state_TRI(mdp,is,MKZAction,0.75);

In [ ]:

CurrentState=relaxed_initial_state_TRI(mdp,CurrentState,MKZAction,0.75)

In [ ]:
string(CurrentState.cars[1].x)

In [ ]:
MLState

In [ ]:
function SendState(NextState::MLState)
    CurrentState=relaxed_initial_state_TRI(mdp,NextState,MKZAction);
    return CurrentState
end

In [ ]:
map(SendState,timestamps)